# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources in the README.md file
- Happy learning!

In [2]:
# Import your libraries:

import numpy as np
import pandas as pd
import warnings
warnings.simplefilter('ignore')

from sklearn.ensemble import RandomForestRegressor as RFR

from sklearn.model_selection import train_test_split as tts

# Challenge 1 - Explore The Data

This lesson will explore the creation of a machine learning pipeline from beggining to end. We will save our model and use the model to make predictions on data outside of our training sample. Let's start by loading the dataset which can be obtained from [Kaggle](https://www.kaggle.com/uciml/mushroom-classification) and [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/mushroom).

In [3]:
# Loading the data

mushrooms = pd.read_csv('../mushrooms.csv')

This dataset contains information about different types of mushrooms. Our response variable indicates whether a mushroom is poisonous. 

####  We will create a model to predict whether a mushroom is poisonous (the `class` column) using the information in all other columns.

Let's print the `head()` of this dataset to see what columns we have in our data.

In [85]:
mushrooms.head()


,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g


#### It looks like the columns in this dataset are coded. 

Let's examine the column types using `dtypes` to confirm this. 

In [5]:
mushrooms.dtypes


class                       object
cap-shape                   object
cap-surface                 object
cap-color                   object
bruises                     object
odor                        object
gill-attachment             object
gill-spacing                object
gill-size                   object
gill-color                  object
stalk-shape                 object
stalk-root                  object
stalk-surface-above-ring    object
stalk-surface-below-ring    object
stalk-color-above-ring      object
stalk-color-below-ring      object
veil-type                   object
veil-color                  object
ring-number                 object
ring-type                   object
spore-print-color           object
population                  object
habitat                     object
dtype: object

In this dataset, each column (feature) of the mushroom is represented by a single-character code. It would be best if we can obtain a dictionary of these codes so that we know what possible codes each column has and what each code represents. This dictionary can be obtained from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/mushroom). In the table below, we print the code dictionary for you to reference.

| Features | Codes |
|---|---|
| classes | edible=e, poisonous=p |
| cap-shape | bell=b,conical=c,convex=x,flat=f,knobbed=k,sunken=s |
| cap-surface | fibrous=f,grooves=g,scaly=y,smooth=s |
| cap-color | brown=n,buff=b,cinnamon=c,gray=g,green=r,pink=p,purple=u,red=e,white=w,yellow=y |
| bruises | bruises=t,no=f |
| odor | almond=a,anise=l,creosote=c,fishy=y,foul=f,musty=m,none=n,pungent=p,spicy=s |
| gill-attachment | attached=a,descending=d,free=f,notched=n |
| gill-spacing | close=c,crowded=w,distant=d |
| gill-size | broad=b,narrow=n |
| gill-color | black=k,brown=n,buff=b,chocolate=h,gray=g, green=r,orange=o,pink=p,purple=u,red=e,white=w,yellow=y |
| stalk-shape | enlarging=e,tapering=t |
| stalk-root | bulbous=b,club=c,cup=u,equal=e,rhizomorphs=z,rooted=r,missing=? |
| stalk-surface-above-ring | fibrous=f,scaly=y,silky=k,smooth=s |
| stalk-surface-below-ring | fibrous=f,scaly=y,silky=k,smooth=s |
| stalk-color-above-ring | brown=n,buff=b,cinnamon=c,gray=g,orange=o,pink=p,red=e,white=w,yellow=y |
| stalk-color-below-ring | brown=n,buff=b,cinnamon=c,gray=g,orange=o,pink=p,red=e,white=w,yellow=y |
| veil-type | partial=p,universal=u |
| veil-color | brown=n,orange=o,white=w,yellow=y |
| ring-number | none=n,one=o,two=t |
| ring-type | cobwebby=c,evanescent=e,flaring=f,large=l,none=n,pendant=p,sheathing=s,zone=z |
| spore-print-color | black=k,brown=n,buff=b,chocolate=h,green=r,orange=o,purple=u,white=w,yellow=y |
| population | abundant=a,clustered=c,numerous=n,scattered=s,several=v,solitary=y |
| habitat | grasses=g,leaves=l,meadows=m,paths=p,urban=u,waste=w,woods=d


SyntaxError: EOF while scanning triple-quoted string literal (<ipython-input-44-bbac330da213>, line 25)

**The columns in the mushrooms dataset seem to be correlated**. We suspect this because many columns seem to have dependencies. For example, if `ring-number` is `n` (none), `ring-type` must be `n` (none) too. Only when `ring-number` is `o` (one) or `t` (two) `ring-type` can be values other than `n`. Other columns may have inter-dependencies too but we can't tell right now. 

Why are we concerned about the variable correlations (also called *multicollinearity*)? It's because if they are strongly correlated, we shouldn't use the linear regression algorithm as the machine learning model. The prediction of the linear regression model is known to be unstable and inaccurate if the variables are strongly correlated. You can read more about why [here](https://en.wikipedia.org/wiki/Multicollinearity#Consequences_of_multicollinearity).

So far it's just our suspicion that the variables are strongly correlated. We need to verify that. Since all the columns are categorical, we cannot use a correlation matrix to examine the degrees of correlations. There are several ways to test whether categorical variables are correlated. The first way is to convert the categorical values to ordinal, then calcualte the correlation matrix with the numerical values. The second way is to use a [Chi-Square test of independence](https://onlinecourses.science.psu.edu/stat500/node/56/) to find out whether there is correlation between each pair of variables. We will use the second way in this lab. So read the explanation of Chi-Square test of independence to understand what it does.

We will begin with the first 2 variables (`cap-shape` and `cap-surface`). Our **null hypothesis of the Chi-Square test of independence is that the two variables are independent** and the **alternative hypothesis is that they are not independent**. To perform Chi-Squre test of independence, we first create a contingency table of those two features using Pandas' [`crosstab`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.crosstab.html) function, then use Scipy's [`chi2_contingency`](https://docs.scipy.org/doc/scipy-0.15.1/reference/generated/scipy.stats.chi2_contingency.html) to test the variable independence from the contingency table.

#### In the following cell, create a contingency table of `cap-surface` and `cap-shape`. Assign this table to the variable `ct`.

In [25]:
ct = pd.crosstab(mushrooms['cap-surface'],mushrooms['cap-shape'])
ct 

cap-shape,b,c,f,k,s,x
cap-surface,,,,,,
f,52,0,1016,60,32,1160
g,1,1,1,1,0,0
s,244,0,820,418,0,1074
y,155,3,1315,349,0,1422


Let's import the chi quare test:

In [12]:
from scipy.stats import chi= np.array(ct)


In the following cell perform the chi square test on the contingency table using the function you just imported. This function wil return a tuple with 4 values. The second value in the tuple is the p-value for our test. Print the p-value and interpret the result - is the null hypothesis rejected? 

In [27]:
chi2_contingency(ct)[1]


4.635777687474967e-206

**No son independientes**

## Now let's create a p-value matrix for all variables.

In the cell below, create a 2-dimensional matrix of all pairwise tests for each two features. Print the matrix and interpret the results.

Below is an example of what your matrix should look like:
![corr df](../corr_df.png)

*Hint: Use loop inside loop to perform pairwise tests for each two features.*

In [42]:
dfc = pd.DataFrame(index = mushrooms.columns, columns = mushrooms.columns)


In [40]:
dfc = pd.DataFrame(index = mushrooms.columns, columns = mushrooms.columns)

for i in mushrooms.columns:
    for j in mushrooms.columns:
        dfc[i][j] = chi2_contingency(pd.crosstab(mushrooms[i],mushrooms[j]))[1]


In [41]:
dfc

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
class,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
cap-shape,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
cap-surface,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
cap-color,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
bruises,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
odor,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
gill-attachment,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
gill-spacing,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
gill-size,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
gill-color,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# Your comment here

We verified the variables are highly correlated from the pairwise Chi-Square test of independence.

#### The next step in model generation is to ensure there is no missing data and handle any missing data if they exist.

In the next cell, check to see if there is any missing data in each column of the dataset

In [47]:
mushrooms.isnull().sum()


class                       0
cap-shape                   0
cap-surface                 0
cap-color                   0
bruises                     0
odor                        0
gill-attachment             0
gill-spacing                0
gill-size                   0
gill-color                  0
stalk-shape                 0
stalk-root                  0
stalk-surface-above-ring    0
stalk-surface-below-ring    0
stalk-color-above-ring      0
stalk-color-below-ring      0
veil-type                   0
veil-color                  0
ring-number                 0
ring-type                   0
spore-print-color           0
population                  0
habitat                     0
dtype: int64

#### Since there is no work to be done to clean up missing data, the next step is to create dummy variables. 

Most machine learning algorithms cannot work with non-numeric data, so we will need to transform our data. Use the [`get_dummies` function](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html) to transform the data. Make sure to remove one dummy column per variable using the `drop_first=True` option.

In [98]:
musdum = pd.get_dummies(mushrooms, drop_first=True)
musdum[1:1]

,class_p,cap-shape_c,cap-shape_f,cap-shape_k,cap-shape_s,cap-shape_x,cap-surface_g,cap-surface_s,cap-surface_y,cap-color_c,...,population_n,population_s,population_v,population_y,habitat_g,habitat_l,habitat_m,habitat_p,habitat_u,habitat_w


#### Our final data exploration task is to prepare the data for modeling by splitting it to predictor, response, train and test. 

We will do this using the [`train_test_split` function from scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html). In the cell below, split the data to `X_train`, `X_test`, `y_train`, and `y_test` using this function. Select 80% of the data for the training sample and the rest for the test sample.

In [96]:
from sklearn.model_selection import train_test_split as tts
y=musdum['class_p']
X=musdum.drop("class_p", axis= 1)

X_train, X_test, y_train, y_test = tts(X, y, test_size= .2)

# Challenge 2 - Creating and Saving Our Model

Determining whether a mushroom is poisonous is a classification task. There are multiple classification models we can choose from.
However, since we have determined that there are many columns that are not indepdendent, this limits our choice of model. One model we will not consider is logistic regression. Two potential choices for this modeling task are [random forest](https://en.wikipedia.org/wiki/Random_forest) and [SVM](https://en.wikipedia.org/wiki/Support_vector_machine).

Let's start with Random Forest. We think of random forest as a voting algorithm. We generate many decision trees by sampling both rows and columns in our dataset. Each one of these trees produces a decision. We let all the trees "vote" and the aggregate decision that they produce gives us the final decision for our algorithm (in this case, they will vote whether each mushroom is poisonous or edible). To learn more about random forests, click [here](https://onlinecourses.science.psu.edu/stat857/node/179/).

In the cell below, we will import and initialize a random forest from scikit-learn ([documentation](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)). Assign the initialized model to `mushroom_rf`. For now, we will just use the default settings for the random forest classifier, so there is no need to pass any arguments to the function.

In [102]:
from sklearn.ensemble import RandomForestClassifier as RFC

mushroom_rf = RFC()


In the cell below, fit the model to the training data.

In [103]:
mushroom_rf.fit(X_train, y_train)


RandomForestClassifier()

#### Next, let's evaluate the model. One of the most straightforward ways to evaluate a classification model is using a confusion matrix. 

The confusion matrix shows us the true positives, false positives, false negatives and true negatives in the data. Our goal is to maximize the true positives and true negatives (the observations that are correctly classified) and minimize the false positives and false negatives.

In the cell below, we'll start by generating predictions for the test data using the `predict` function. 

In [104]:
y_pred = mushroom_rf.predict(X_test)


Now we'll import the `confusion_matrix` function ([documentation](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html)) and compute the [confusion matrix](https://en.wikipedia.org/wiki/Confusion_matrix) by comparing the observed data (`y_test`) and the predicted data that you found in the cell above.

In [105]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_pred)


array([[811,   0],
       [  0, 814]])

# Bonus Challenge 1 - Use a Different ML Algorithm to Evaluate Model

Repeat the steps here to predict and evaluate the model but instead use gradient boosted classification. Your end result should be a confusion matrix comparing the predicted and observed y values for the test sample. You can read more about gradient boosting [here](http://blog.kaggle.com/2017/01/23/a-kaggle-master-explains-gradient-boosting/).

To calculate the gradient boosted classfication, use the [`GradientBoostingClassifier` of Scikit-Learn](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html).

In [81]:
from sklearn.ensemble import GradientBoostingClassifier as GBC
gb = GBC()
gb.fit(X_train, y_train.array)
y_pred = gb.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[814,   0],
       [  2, 809]])

# Challenge 3 - Producing Individual Predictions and Saving The Model

One of the most important goals of machine learning models is to act as something like a prediction black box. We would like to pass an observation to the model and get back a prediction as an output. Let's do this in the next cells using the `predict` function. What we want to do is to pick a random mushroom and generate a prediction that will tell us whether it is poisonous.

#### In the next cell, create a function called `get_random_data` that accepts a dataset and returns a random row from the dataset.

In [92]:
from numpy import random
from random import seed
from random import randint

def get_random_data(dataset):
    r = random.randint(len(dataset))
    
    """
    Returns a random row of a dataset
    
    Args:
        dataset [dataframe]: a Pandas dataframe containing a dataset
    
    Returns:
        A random row in the dataset
    """
    # Your code here
    return dataset[r:r+1]

#### Call `get_random_data` to obtain a random row from the test dataset. Assign the returned row to `random_mushroom` and print.

In [99]:
random_mushroom = get_random_data(musdum)
random_mushroom

,class_p,cap-shape_c,cap-shape_f,cap-shape_k,cap-shape_s,cap-shape_x,cap-surface_g,cap-surface_s,cap-surface_y,cap-color_c,...,population_n,population_s,population_v,population_y,habitat_g,habitat_l,habitat_m,habitat_p,habitat_u,habitat_w
2105,0,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


#### In the cell below, use the `predict` function to generate a prediction for this random mushroom. Is the random mushroom poisonous? Compare this to the true y value.

In [100]:
y_rnd_pred = RndF.predict(random_mushroom.drop("class_p", axis= 1))
y_rnd_pred

array([0], dtype=uint8)

**Si es 0 es que no es P por lo cual no es venenosa**

#### Our final step is to save our model. 

Do this in the cell below using [pickling](https://docs.python.org/3/library/pickle.html). Import the pickle library and save the `mushroom_rf` model as a pickle file using `pickle.dump()`. Name your file `mushrooms.sav`

In [116]:
import pickle
filename = "mushrooms.sav"
pickle.dump(mushroom_rf, open(filename, 'wb'))

'''loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test, Y_test)
print(result)'''

"loaded_model = pickle.load(open(filename, 'rb'))\nresult = loaded_model.score(X_test, Y_test)\nprint(result)"

## Bonus Challenge 2 - Heatmap to Visualize Data Correlation

Practice generating a heatmap to visualize the pairwise column correlations. You can do one of the following two things (or both if you like):

* Create a heatmap using the p-value matrix for the pairwise Chi-Square test of independence. 

* Use Scikit-Learn's `LabelEncoder` ([documentation](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html)) to transform the raw categorical variables to ordinal. Then calculate the correlation matrix using the ordinal values.

If you're scratching your hairs off, refer to [this example for the first way](https://towardsdatascience.com/the-search-for-categorical-correlation-a1cf7f1888c9) and [this example for the second way](https://www.kaggle.com/haimfeld87/analysis-and-classification-of-mushrooms). 

/bin/bash: wd: command not found
